In [1]:
#Code for central or centralv2
#Can be used just by switching the csv file path
#the cell below is used for centralv2 as an example

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Central Codes (Can be used for Central or Centralv2)

# Path to shapefile
shapefile_path = 'drive-download-2025countyoutlines-1-001.zip' 

# Path to CSV (central restrict v2)
csv_path = 'county_results_central.restrict.v2 - county_results_central.restrict.v2.csv' 

# Year Filter (choose whichever year)
selected_year = 2035

# Output file
output_geojson = f"biomass_by_county_{selected_year}.geojson"

# For Loading County Shapefile

gdf = gpd.read_file(shapefile_path)
# just drop the missing FIPS in case
gdf = gdf[gdf['fips'].notnull()]

# Convert from float to int to str.zfill(5) (because it wonn't compute the data) 
gdf['fips'] = gdf['fips'].astype(float).astype(int).astype(str).str.zfill(5)


print(f"✅ Loaded shapefile with {len(gdf)} counties.")

#Load Biomass Data

df = pd.read_csv(csv_path)

# Ensure FIPS is string with 5 digits
df['fips'] = df['fips'].astype(str).str.zfill(5)

# Filter by selected year
df = df[df['year'] == selected_year]

if df.empty:
    raise ValueError(f"No data found for year {selected_year}")

print(f"✅ Loaded {len(df)} biomass records for year {selected_year}.")

# Aggregate Biomass Data (maybe fix because it could be cause of why all the data looks the same?)

biomass_summary = df.groupby('fips').agg({
    'tons_of_production': 'sum',
    'production_acres': 'sum',
    'land_use_factor': 'mean' 
}).reset_index()

print(f"✅ Aggregated biomass data for {len(biomass_summary)} counties.")

# Merge With County Shapes

gdf_merged = gdf.merge(biomass_summary, on='fips', how='left')

# how many counties received data
num_with_data = gdf_merged['tons_of_production'].notnull().sum()
print(f"✅ Joined data: {num_with_data} counties have biomass values.")

# Export to GeoJSON 

gdf_merged.to_file(output_geojson, driver='GeoJSON')
print(f"✅ Exported merged GeoDataFrame to: {output_geojson}")

# Plot Map 

fig, ax = plt.subplots(1, 1, figsize=(15, 10))

gdf_merged.plot(
    column='tons_of_production',
    cmap='YlOrRd',
    linewidth=0.1,
    ax=ax,
    edgecolor='black',
    legend=True,
    missing_kwds={"color": "lightgrey", "label": "No data"}
)

ax.set_title(f"Tons of Biomass Production by County ({selected_year})", fontsize=16)
ax.axis('off')
plt.tight_layout()
plt.show()


✅ Loaded shapefile with 3107 counties.
✅ Loaded 37623 biomass records for year 2035.
✅ Aggregated biomass data for 3108 counties.
✅ Joined data: 3101 counties have biomass values.
✅ Exported merged GeoDataFrame to: biomass_by_county_2035.geojson
